In [1]:
import tensorflow as tf
import numpy as np

In [ ]:
[67.99964  34.001972 27.370667 39.080585 -1.       -1.       -1.
  -1.       -1.       -1.       -1.       -1.       -1.       -1.
  -1.       -1.       -1.       -1.       -1.       -1.       51.07086
  65.03794  -1.       -1.       -1.       -1.       -1.       -1.
  50.506577 79.14521 ]
[-1.1004776   0.6365961   0.7179315   1.483962    0.03935888 -1.2409312
   1.271584   -0.17267627  1.7933336  -2.56411     0.8744608  -0.3783334
   2.1389403   1.2774602  -0.61297405  0.4473193   2.032928    1.7558423
   0.83670133  1.6127489  -0.30488694 -0.7636453  -1.5378115   1.7087466
  -1.2598088  -0.46888253 -0.28241813 -1.2659875   0.50864017  0.09317458]